In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import onnx
import onnxruntime as ort
import numpy as np

# Je crée une classe qui représente mon modèle
# Elle hérite de nn.Module qui sert a pytorch pour fonctionné
class CNNModel(nn.Module):
    def __init__(self):
        # J'appelle le constructeur de base
        super(CNNModel, self).__init__()
        #je prépare les images
        # ça va me permettre d'analyser les images d'après ce que j'ai pu comprendre et voir c'est ce que l'on appel une "vision"
        self.conv_layers = nn.Sequential(
            # l'image qui va être passée sera lu par un filtre qui sert à regarder les formes et les motifs
            # le 1 l'image en noir et blanc, 32 le nombre de filtres, kernel_size=3 la taille du filtre, padding=1 pour garder la taille de l'image
            # 28x28 -> 28x28
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  
            # Relu va gardé les valeurs positives et mettre à zéro les négatives
            nn.ReLU(),
            # permet de réduire la taille de l'image par 2 du coup 14x14 il devrait ne garder que les pixels les plus importants 
            nn.MaxPool2d(2),                             
            # Cette partie va continuer  de filtré cette fois-ci avec 64 filtres
            #32 analyse différentes qui va regarder l'image sous différentes formes
            # pour 64 c'est le même principe que pour 32
            nn.Conv2d(32, 64, kernel_size=3, padding=1), 
            # va de nouveau garder les valeurs positives
            nn.ReLU(),
            # Dropout spatial pour les convolutions explication de celui-ci : https://pytorch.org/docs/stable/generated/torch.nn.Dropout2d.html
            # je le trouve intéressent car il permet de supprimer des canaux entiers temporairement  et ça permet au réseau de neurones de ne pas s'appuyer sur 
            #un seul filtre. du coup certains de 32 canaux seront mis à zéro aléatoirement je pense que ce n'est pas pertinent pour l'excercice mais c'est intéressant
            nn.Dropout2d(0.25),  
            # va de nouveau réduire la taille de l'image par 2 pour ne garder que l essentiel
            nn.MaxPool2d(2)                             
        )

        # la deuxième partie et celle qui va transformer les données filtrées en une forme que le modèle peut utiliser pour faire des prédictions
        self.fc_layers = nn.Sequential(
            # va permettre de mettre les données en une seule ligne pour pouvoir les calculer
            nn.Flatten(),
            # va faire un calcul pour réduire les données à 128 valeurs
            nn.Linear(64 * 7 * 7, 128),
            # va de nouveau garder les valeurs positives
            nn.ReLU(),
            # va aider à éviter le sur-apprentissage en mettant à zéro 50% des valeurs aléatoirement
            nn.Dropout(0.5), 
            # va faire un autre calcul pour réduire les données à 10 valeurs (une pour chaque chiffre de 0 à 9)
            nn.Linear(128, 10)
        )
       # Cette fonction dit comment les données passent à travers le modèle
    def forward(self, x):
        # va pemettre de passer l'image et de lire le contenu
        x = self.conv_layers(x)
        # c'est la partie reçoit les données filtrées et les transforme en une forme que le modèle peut utiliser pour faire des prédictions
        x = self.fc_layers(x)
        # retourne les probabilités  
        return x

# je prépare les images en les transformant en nombres 
transform = transforms.Compose([
    #transforme l'image
    transforms.ToTensor(),
    # ajuste des images pour les rendre plus faciles à traiter
    transforms.Normalize((0.1307,), (0.3081,))  
])

# Je charge les données MNIST
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=True, download=True, transform=transform),
    #peermet de regarder 64 images à la fois et shuffle=True pour mélanger les images 
    batch_size=64, shuffle=True
)

# Je charge les données de test MNIST
# batch_size=256 pour regarder 256 images à la fois et shuffle=False pour ne pas mélanger les images
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('.', train=False, download=True, transform=transform),
    batch_size=256, shuffle=False
)

# je crée le modèle
model = CNNModel()
#permet de calculer les erreurs
criterion = nn.CrossEntropyLoss()
# permet de mettre à jour les paramètres du modèle pour améliorer les prédictions
# optim.Adam est un algorithme d'optimisation qui ajuste les paramètres du modèle pour améliorer les prédictions
# lr=0.001 est le taux d'apprentissage, qui détermine à quelle vitesse le modèle apprend
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entraînement
# Je vais entraîner le modèle pendant 5 fois
for epoch in range(5):
    # Je mets le modèle en mode entraînement
    # ça permet de dire au modèle qu'il va apprendre et qu'il doit mettre à jour
    model.train()
    ## ça garde une trace des erreurs pendant l'entraînement
    running_loss = 0
    #récupère les images et les valeurs
    for images, labels in train_loader:
        # remet les gradients à zéro pour éviter de les additionner
        optimizer.zero_grad()
        # passe les images dans le modèle pour obtenir les prédictions
        outputs = model(images)
        #comparaison des valeurs
        loss = criterion(outputs, labels)
        # permet de dire les erreurs au modèle pour qu'il puisse apprendre
        loss.backward()
        # ajuste les paramétres du modèle en fonction des erreurs
        # ça permet de mettre à jour les paramètres du modèle pour améliorer les prédictions
        optimizer.step()
        #ajoute l'erreur au total
        running_loss += loss.item()
        # sa affiche l erreur moyenne  plsu elle diminue plus le modéle apprend
    print(f"Époque {epoch+1} terminée. Perte moyenne = {running_loss/len(train_loader):.4f}")

# permet de passer a la prédiction donc ce n'est plus de l'entrainement je vais pouvoir me rendre si l'entrainement a été efficace
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        #fait des prédictions sur les images 
        outputs = model(images)
        # ne garde que la valeur la plus proche de la prédiction
        predicted = torch.argmax(outputs, 1)
        # c'est le nombre total d'images
        total += labels.size(0)
        # le nombre de prédictions correctes
        correct += (predicted == labels).sum().item()
        #affiche le taux de réussite
print(f"Précision sur les données de test : {100 * correct / total:.2f}%")

# exportation vers ONNX
# je crée une entrée factice pour le modèle
# ça permet de simuler une image pour l'exportation
dummy_input = torch.randn(1, 1, 28, 28)
# je spécifie le nom du fichier ONNX
onnx_file = "web/mnist_model_cnn.onnx"
# J'exporte le modèle vers le format ONNX
# ça permet de sauvegarder le modèle pour l'utiliser dans d'autres applications
torch.onnx.export(model, dummy_input, onnx_file,
                  input_names=['input'], output_names=['output'],
                  dynamic_axes={'input': {0: 'batch'}, 'output': {0: 'batch'}})
print(f"Modèle CNN exporté en ONNX dans '{onnx_file}'")

# Test ONNX
# Je charge le modèle ONNX
# ça permet de charger le modèle ONNX pour faire des prédictions
ort_session = ort.InferenceSession(onnx_file)
# Je prends une image de test pour faire une prédiction
test_image = next(iter(test_loader))[0][0].unsqueeze(0).numpy()
# ça permet de préparer l'image pour la prédiction , float32 pour que les valeurs soient en virgule flottante
inputs = {"input": test_image.astype(np.float32)}
# Je fais une prédiction avec le modèle ONNX
outputs = ort_session.run(None, inputs)
# Je prends la valeur la plus proche de la prédiction
# ça permet de savoir quel chiffre le modèle pense que c'est
prediction = np.argmax(outputs[0])
# Affichage de la prédiction
print("Test ONNX : prédiction =", prediction)



Époque 1 terminée. Perte moyenne = 0.1329
Époque 2 terminée. Perte moyenne = 0.0407
Époque 3 terminée. Perte moyenne = 0.0294
Époque 4 terminée. Perte moyenne = 0.0206
Époque 5 terminée. Perte moyenne = 0.0177
Précision sur les données de test : 99.07%
Modèle CNN exporté en ONNX dans 'web/mnist_model_cnn.onnx'
Test ONNX : prédiction = 7
